In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi
!cat /proc/cpuinfo
!rm -rf sample_data

# !git clone --recursive https://github.com/Microsoft/LightGBM.git
# !cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds 

In [ ]:
!mkdir data
# !cp -r /content/drive/MyDrive/Data-x/data/*.csv /content/data/
# !cp -r /content/drive/MyDrive/Data-x/data/0329/*.csv /content/data/
# !cp -r /content/drive/MyDrive/Data-x/data/0330/*.csv /content/data/
!cp /content/drive/MyDrive/Data-x/data/filtered/*.csv /content/data/

In [ ]:
%cd /content

/content


In [ ]:
%cd data
!for file in *; do mv "${file}" "${file/_filtered/}"; done

/content/data
mv: '5cd56b5ae2acfd2d33b58544_test.csv' and '5cd56b5ae2acfd2d33b58544_test.csv' are the same file
mv: '5cd56b5ae2acfd2d33b58544_train.csv' and '5cd56b5ae2acfd2d33b58544_train.csv' are the same file
mv: '5cd56b5ae2acfd2d33b58546_test.csv' and '5cd56b5ae2acfd2d33b58546_test.csv' are the same file
mv: '5cd56b5ae2acfd2d33b58546_train.csv' and '5cd56b5ae2acfd2d33b58546_train.csv' are the same file
mv: '5cd56b5ae2acfd2d33b58548_test.csv' and '5cd56b5ae2acfd2d33b58548_test.csv' are the same file
mv: '5cd56b5ae2acfd2d33b58548_train.csv' and '5cd56b5ae2acfd2d33b58548_train.csv' are the same file
mv: '5cd56b5ae2acfd2d33b58549_test.csv' and '5cd56b5ae2acfd2d33b58549_test.csv' are the same file
mv: '5cd56b5ae2acfd2d33b58549_train.csv' and '5cd56b5ae2acfd2d33b58549_train.csv' are the same file
mv: '5cd56b5ae2acfd2d33b5854a_test.csv' and '5cd56b5ae2acfd2d33b5854a_test.csv' are the same file
mv: '5cd56b5ae2acfd2d33b5854a_train.csv' and '5cd56b5ae2acfd2d33b5854a_train.csv' are the same f

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path
import glob

from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import psutil
import random
import os
import time
import sys
import math
from contextlib import contextmanager


N_SPLITS = 10
SEED = 42

@contextmanager
def timer(name: str):
    t0 = time.time()
    p = psutil.Process(os.getpid())
    m0 = p.memory_info()[0] / 2. ** 30
    try:
        yield
    finally:
        m1 = p.memory_info()[0] / 2. ** 30
        delta = m1 - m0
        sign = '+' if delta >= 0 else '-'
        delta = math.fabs(delta)
        print(f"[{m1:.1f}GB({sign}{delta:.1f}GB): {time.time() - t0:.3f}sec] {name}", file=sys.stderr)


def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]

set_seed(SEED)

feature_dir = './'
train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train.csv')))
print(train_files)
test_files = sorted(glob.glob(os.path.join(feature_dir, '*_test.csv')))

['./5cd56b5ae2acfd2d33b58544_train.csv', './5cd56b5ae2acfd2d33b58546_train.csv', './5cd56b5ae2acfd2d33b58548_train.csv', './5cd56b5ae2acfd2d33b58549_train.csv', './5cd56b5ae2acfd2d33b5854a_train.csv', './5d27075f03f801723c2e360f_train.csv', './5d27096c03f801723c31e5e0_train.csv', './5d27097f03f801723c320d97_train.csv', './5d27099f03f801723c32511d_train.csv', './5d2709a003f801723c3251bf_train.csv', './5d2709b303f801723c327472_train.csv', './5d2709bb03f801723c32852c_train.csv', './5d2709c303f801723c3299ee_train.csv', './5d2709d403f801723c32bd39_train.csv']


# Baseline 1: Ridge Regression

In [ ]:
data = pd.read_csv(train_files[0], index_col=0)
test_data = pd.read_csv(test_files[0], index_col=0)

wifi = data.iloc[:,:-6]
x = data.iloc[:,-6:-5]
y =data.iloc[:,-5:-4]
floor = data.iloc[:,-4:-3]

wifi_val = test_data.iloc[:,:-6]
x_val = test_data.iloc[:,-6:-5]
y_val =test_data.iloc[:,-5:-4]
floor_val = test_data.iloc[:,-4:-3]

print(test_data.iloc[:,-6:-5])

            x
4   23.492540
4   28.287237
4   31.521397
4   28.287237
4   31.521397
..        ...
4   24.734066
4   37.727207
4   32.622036
4   20.402344
4   15.472065

[181 rows x 1 columns]


In [ ]:
ridge_model_x = Ridge(alpha=100, fit_intercept=True)
ridge_model_x.fit(wifi, x)

ridge_model_y = Ridge(alpha=100, fit_intercept=True)
ridge_model_y.fit(wifi, y)

ridge_model_f = Ridge(alpha=100, fit_intercept=True)
ridge_model_f.fit(wifi, floor)

Ridge(alpha=100, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [ ]:
ridge_train_error_x = mean_squared_error(ridge_model_x.predict(wifi), x)
ridge_train_error_y = mean_squared_error(ridge_model_y.predict(wifi), y)
ridge_train_error_f = mean_squared_error(ridge_model_f.predict(wifi), floor)
print("Ridge Regression train MSE: ", ridge_train_error_x, ridge_train_error_y, ridge_train_error_f)

ridge_test_error_x_val = mean_squared_error(ridge_model_x.predict(wifi_val), x_val)
ridge_test_error_y_val = mean_squared_error(ridge_model_y.predict(wifi_val), y_val)
ridge_test_error_f_val = mean_squared_error(ridge_model_f.predict(wifi_val), floor_val)
print("Ridge Regression test MSE: ", ridge_test_error_x_val, ridge_test_error_y_val, ridge_test_error_f_val)

Ridge Regression train MSE:  35.59955988257892 128.36572145533978 0.040629434052643756
Ridge Regression test MSE:  48.43326117360577 163.25574238550809 0.05223570157218679


# Baseline 2

In [ ]:
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, validation_curve, cross_val_score, GridSearchCV
from matplotlib import pyplot as plt
from timeit import default_timer as timer
from sklearn import linear_model

clf_x = linear_model.Lasso(alpha=0.1, max_iter = 10000)
clf_y = linear_model.Lasso(alpha=0.1, max_iter = 10000)
clf_f = linear_model.Lasso(alpha=0.1, max_iter = 10000)

clf_x.fit(wifi, x)
clf_y.fit(wifi, y)
clf_f.fit(wifi, floor)

lasso_train_error_x = mean_squared_error(clf_x.predict(wifi), x)
lasso_train_error_y = mean_squared_error(clf_y.predict(wifi), y)
lasso_train_error_f = mean_squared_error(clf_f.predict(wifi), floor)
print("Lasso Regression train MSE: ", lasso_train_error_x, lasso_train_error_y, lasso_train_error_f)

lasso_test_error_x_val = mean_squared_error(clf_x.predict(wifi_val), x_val)
lasso_test_error_y_val = mean_squared_error(clf_y.predict(wifi_val), y_val)
lasso_test_error_f_val = mean_squared_error(clf_f.predict(wifi_val), floor_val)
print("Lasso Regression test MSE: ", lasso_test_error_x_val, lasso_test_error_y_val, lasso_test_error_f_val)

Lasso Regression train MSE:  35.599604529407266 128.36575773378826 0.040672530740461135
Lasso Regression test MSE:  48.399159983600164 163.18798901796495 0.051147917293936564


In [ ]:
print("Ridge Regression train MSE: ", ridge_train_error_x, ridge_train_error_y, ridge_train_error_f)
print("Ridge Regression test MSE: ", ridge_test_error_x_val, ridge_test_error_y_val, ridge_test_error_f_val)

print("Lasso Regression train MSE: ", lasso_train_error_x, lasso_train_error_y, lasso_train_error_f)
print("Lasso Regression test MSE: ", lasso_test_error_x_val, lasso_test_error_y_val, lasso_test_error_f_val)

Ridge Regression train MSE:  35.59955988257892 128.36572145533978 0.040629434052643756
Ridge Regression test MSE:  48.43326117360577 163.25574238550809 0.05223570157218679
Lasso Regression train MSE:  35.599604529407266 128.36575773378826 0.040672530740461135
Lasso Regression test MSE:  48.399159983600164 163.18798901796495 0.051147917293936564
